# Defining data types

We can define types (i.e. data structures) ourselves using the `struct` keyword.

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case).

(Note that types can not be redefined - you have to restart your Julia session to change a type definiton.)

In [2]:
struct MyType end

To create an object of type `MyType` we have to call a [constructor](https://docs.julialang.org/en/latest/manual/constructors/). Loosely speaking, a constructor is a function that create new objects.

Julia automatically creates a trivial constructors for us, which has the same name as the type.

In [53]:
methods(MyType)

# 1 method for generic function "(::Type)":
[1] MyType() in Main at In[2]:1

In [3]:
m = MyType()

MyType()

In [4]:
typeof(m)

MyType

In [5]:
m isa MyType

true

Since no data is contained in our `MyType`  - it is a so-called *singleton type* - we can basically only use it for dispatch.

In [10]:
f(x::MyType) = println("I'm the only method that works with objects of MyType.")

f (generic function with 1 method)

In [11]:
f(m)

I'm the only method that works with objects of MyType.


Most of the time, we'll want a self-defined type to hold some data. For this, we need *fields*.

In [13]:
struct A
    x::Int64
end

In [14]:
A()

MethodError: MethodError: no method matching A()
Closest candidates are:
  A(!Matched::Int64) at In[13]:2
  A(!Matched::Any) at In[13]:2

The default constructor always expects values for all fields.

In [15]:
A(3)

A(3)

In [16]:
a = A(3)

A(3)

In [17]:
# a.<TAB>
a.x

3

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [21]:
a.x = 2

ErrorException: setfield! immutable struct of type A cannot be changed

In [22]:
mutable struct B
    x::Int64
end

In [23]:
b = B(3)

B(3)

In [24]:
b.x

3

In [25]:
b.x = 4

4

In [26]:
b.x

4

Note, however, that **immutability is not recursive**.

In [159]:
struct C
    x::Vector{Int64}
end

In [160]:
c = C([1, 2, 3])

C([1, 2, 3])

In [161]:
c.x

3-element Array{Int64,1}:
 1
 2
 3

In [162]:
c.x = [3,4,5]

ErrorException: setfield! immutable struct of type C cannot be changed

In [163]:
c.x[1] = 3

3

In [164]:
c.x

3-element Array{Int64,1}:
 3
 2
 3

In [165]:
c.x .= [3,4,5] # dot to perform the assignment element-wise

3-element Array{Int64,1}:
 3
 4
 5

Abstract types are just as easy to define using the keyword `abstract type`.

In [34]:
abstract type MyAbstractType end

Since abstract types don't have fields, they only (informally) define interfaces and can be used for dispatch.

In [35]:
struct MyConcreteType <: MyAbstractType # subtype
    somefield::String
end

In [38]:
c = MyConcreteType("test")

MyConcreteType("test")

In [39]:
c isa MyAbstractType

true

In [42]:
supertype(MyConcreteType)

MyAbstractType

In [44]:
subtypes(MyAbstractType)

1-element Array{Any,1}:
 MyConcreteType

# Custom constructor

In [55]:
struct VolNaive
    value::Float64
end

In [56]:
VolNaive(3.0)

VolNaive(3.0)

In [57]:
VolNaive(-3.0)

VolNaive(-3.0)

In [65]:
struct VolSimple
    value::Float64
    
    function VolSimple(x) # inner constructor. function name must match the type.
        if !(x isa Real)
            throw(ArgumentError("Must be real"))
        end
        if x < 0
            throw(ArgumentError("Negative volume not allowed."))
        end
        new(x) # within an inner constructor, the `new` function can be used to create an object.
    end
end

---

**Side note:**

```julia
if !(x isa Real)
    throw(ArgumentError("Must be real"))
end
if x < 0
    throw(ArgumentError("Negative volume not allowed."))
end
```

This can be written more compactly as
```julia
x isa Real || throw(ArgumentError("Must be real"))
x < 0 && throw(ArgumentError("Negative volume not allowed."))
```

See ["short-circuit evaluation"](https://docs.julialang.org/en/latest/manual/control-flow/#Short-Circuit-Evaluation-1) for more information.

---

In [70]:
VolSimple(3.0)

VolSimple(3.0)

In [71]:
VolSimple(-3.0)

ArgumentError: ArgumentError: Negative volume not allowed.

In [72]:
VolSimple("test")

ArgumentError: ArgumentError: Must be real

In [69]:
VolSimple(3) # implicit conversion from Int64 -> Float64

VolSimple(3.0)

# Parametric types

Volumes don't have to be `Float64` values. We can easily relax our type definition to allow all sorts of internal value types.

In [77]:
struct VolParam{T}
    value::T
    
    function VolParam(x::T) where T # x can be of any type T
        if !(x isa Real)
            throw(ArgumentError("Must be real"))
        end
        if x < 0
            throw(ArgumentError("Negative volume not allowed."))
        end
        new{T}(x) # Note that we need an extra {T} here
    end
end

In [79]:
VolParam(3.0)

VolParam{Float64}(3.0)

In [80]:
VolParam(3)

VolParam{Int64}(3)

Instead of checking the realness of the input `x` explicitly in the inner constructor, we can impose type constraints in the type and function signatures.

In [1]:
struct Vol{T<:Real} <: Real # the last <: Real tells Julia that a Vol is a subtype of Real, i.e. basically a real number
    value::T
    
    function Vol(x::T) where T<:Real # x can be of any type T<:Real
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new{T}(x)
    end
end

In [2]:
Vol(3)

Vol{Int64}(3)

In [3]:
Vol(3.0)

Vol{Float64}(3.0)

In [4]:
Vol("1.23")

MethodError: MethodError: no method matching Vol(::String)
Closest candidates are:
  Vol(!Matched::Complex) where T<:Real at complex.jl:37
  Vol(!Matched::T<:Real) where T<:Real at In[1]:5
  Vol(!Matched::T<:Number) where T<:Number at boot.jl:741
  ...

In [5]:
Vol(-2)

ArgumentError: ArgumentError: Negative volume not allowed.

# Arithmetic

In [6]:
Vol(3) + Vol(4)

ErrorException: + not defined for Vol{Int64}

In [7]:
+(x::Vol, y::Vol) = Vol(x.value + y.value)

ErrorException: error in method definition: function Base.+ must be explicitly imported to be extended

If we want to extend or override functions that already exit, we need to `import` them first.

In [8]:
import Base: +

+(x::Vol, y::Vol) = Vol(x.value + y.value)

+ (generic function with 164 methods)

In [9]:
Vol(3) + Vol(4)

Vol{Int64}(7)

In [10]:
Vol(2) + Vol(8.3) # implicit conversion!

Vol{Float64}(10.3)

In [11]:
methodswith(Vol)

[1] +(x::Vol, y::Vol) in Main at In[8]:3

In [12]:
import Base: -, *

-(x::Vol, y::Vol) = Vol(x.value - y.value)
*(x::Vol, y::Vol) = Vol(x.value * y.value)

* (generic function with 347 methods)

Now that we have addition defined for our volume type, some functions already **just work**.

In [13]:
sum([Vol(3), Vol(4.8), Vol(1)])

Vol{Float64}(8.8)

In [14]:
M = Vol.(rand(3,3))

3×3 Array{Vol{Float64},2}:
 Vol{Float64}(0.433303)  Vol{Float64}(0.644252)  Vol{Float64}(0.73798) 
 Vol{Float64}(0.730542)  Vol{Float64}(0.052614)  Vol{Float64}(0.285831)
 Vol{Float64}(0.70091)   Vol{Float64}(0.394457)  Vol{Float64}(0.261248)

In [15]:
N = Vol.(rand(3,3))

3×3 Array{Vol{Float64},2}:
 Vol{Float64}(0.375645)  Vol{Float64}(0.0342596)  Vol{Float64}(0.614994)
 Vol{Float64}(0.897213)  Vol{Float64}(0.776969)   Vol{Float64}(0.979462)
 Vol{Float64}(0.270326)  Vol{Float64}(0.788167)   Vol{Float64}(0.193124)

In [16]:
M + N

3×3 Array{Vol{Float64},2}:
 Vol{Float64}(0.808948)  Vol{Float64}(0.678511)  Vol{Float64}(1.35297) 
 Vol{Float64}(1.62775)   Vol{Float64}(0.829583)  Vol{Float64}(1.26529) 
 Vol{Float64}(0.971236)  Vol{Float64}(1.18262)   Vol{Float64}(0.454372)

Whenever something doesn't work, we implement the necessary functions.

In [17]:
sin(Vol(3))

MethodError: MethodError: no method matching AbstractFloat(::Vol{Int64})
Closest candidates are:
  AbstractFloat(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:194
  AbstractFloat(::T<:Number) where T<:Number at boot.jl:741
  AbstractFloat(!Matched::Bool) at float.jl:252
  ...

In [19]:
import Base: AbstractFloat
AbstractFloat(x::Vol{T}) where T = AbstractFloat(x.value)

AbstractFloat

In [20]:
sin(Vol(3))

0.1411200080598672

In [21]:
sqrt(Vol(3))

1.7320508075688772

If we really wanted to have `Vol{T}` objects behave like real numbers in all operations, we'd have to do a bit more work like specifying [promotion and conversion rules](https://docs.julialang.org/en/latest/manual/conversion-and-promotion/).

An important thing to note is that **user defined types are just as good as built-in types**!

There is nothing special about built-in types. In fact, [they are implemented in the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our volume "wrapper" type does not come with any performance overhead by benchmarking it in a simple function.

# Benchmarking with `BenchmarkTools.jl`

In [34]:
using BenchmarkTools

In [35]:
operation(x) = x^2 + sqrt(x)

operation (generic function with 1 method)

In [36]:
x = rand(2,2)
@time operation.(x)

  0.055194 seconds (174.03 k allocations: 8.518 MiB)


2×2 Array{Float64,2}:
 1.24786   0.497051
 0.306763  0.163774

In [37]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

f (generic function with 1 method)

In [38]:
f()

  0.000000 seconds (1 allocation: 112 bytes)


2×2 Array{Float64,2}:
 0.903038  0.81437
 1.41182   1.26867

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [39]:
@benchmark operation.(x)

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  3
  --------------
  minimum time:     317.035 ns (0.00% GC)
  median time:      337.559 ns (0.00% GC)
  mean time:        414.877 ns (7.50% GC)
  maximum time:     209.241 μs (99.80% GC)
  --------------
  samples:          10000
  evals/sample:     229

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [40]:
@btime operation.(x);

  316.522 ns (3 allocations: 144 bytes)


However, we still have to take some care to avoid accessing global variables.

In [41]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

  31.590 ns (1 allocation: 112 bytes)


More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).

Finally, we can check the performance of our custom volume type.

In [50]:
@btime sqrt(Vol(3));
@btime sqrt(3);

  1.299 ns (0 allocations: 0 bytes)
  1.299 ns (0 allocations: 0 bytes)


# Core messages of this Notebook

* There are `mutable struct`s and immutable `struct`s and immutability is not recursive.
* **Contructors** are functions that create objects. In an inner constructor we can use the function `new` to generate objects.
* We can easily **extend `Base` functions** for our types to implement arithmetics and such.
* We should always benchmark our code with **BenchmarkTools.jl's @btime and @benchmark**.

# Exercise: One-hot vector

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is useful in machine learning, as we'll see later.

It simply means that among a group of bits (all either 0 or 1) only one is hot (1) while all others are cold (0),

`v = [0, 0, 0, 0, 0, 1, 0, 0, 0]`

### Task

1. Think about what information an implementation of a one-hot vector actually has to store.
2. Define a `OneHot` type which represents a vector with only a single hot (i.e. `== 1`) bit.
3. Extend all the necessary `Base` functions such that the following computation works for a matrix `A` and a vector of `OneHot` vectors `vs` (i.e. `vs isa Vector{OneHot}`).

    ```julia
    function innersum(A, vs)
        t = zero(eltype(A)) # generic!
        for v in vs
            y = A*v
            for i in 1:length(vs[1])
                t += v[i] * y[i]
            end
        end
        return t
    end

    A = rand(3,3)
    vs = [rand(3) for _ = 1:10] # This should be replaced by a `Vector{OneHot}`

    innersum(A, vs)

    ```

4. Benchmark the speed of `innersum` when called with a `OneHot` vector or with a `Vector{Float64}`, respectively.
 * Do you observe a speed up?


5. Now, define a `OneHotVector` type which is identical to `OneHot` but is declared to be a subtype of `AbstractVector{Bool}` and extend only the functions `Base.getindex(v::OneHotVector, i::Int)` and `Base.size(v::OneHotVector)`.
 * Here, the function `size` should return a `Tuple{Int64}` indicating the length of the vector, i.e. `(3,)` for a one-hot vector of length 3 (see the [AbstractArray interface](https://docs.julialang.org/en/latest/manual/interfaces/#man-interface-array-1) for more information)
 

6. Try to create a single `OneHotVector` and try to run the `innersum` function using the new `OneHotVector` type.
 * What changes do you observe?
 * Do you have to implement any further methods?